In [4]:
from minitrade.backtest import Strategy

class DualMomentumStrategy(Strategy):
    lookback = 100  # 12 months for daily data
    top_n = 3       # Number of instruments to select

    def init(self):
        # Calculate rolling returns for each instrument
        self.returns = self.I(self.data.ta.roc(self.lookback), name='ROC')

    def next(self):
        self.alloc.assume_zero()
        latest_returns = self.returns.df.iloc[-1]
        positive = latest_returns[latest_returns > 0]
        top = positive.sort_values(ascending=False).head(self.top_n)
        n_selected = len(top)
        if n_selected > 0:
            self.alloc.bucket['main'].append(top.index)
            self.alloc.bucket['main'].weight_explicitly(1 / n_selected).apply()
        self.rebalance(cash_reserve=0.01)



In [5]:
from minitrade.datasource import QuoteSource
from minitrade.backtest import Backtest

yahoo = QuoteSource.get_source('Yahoo')
symbols = [
    'MMM', 'AXP', 'AAPL', 'BA', 'CVX', 'CAT', 'CSCO', 'KO', 'DIS', 'DOW',
    'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT'
]
data = yahoo.daily_bar(','.join(symbols), start='2024-01-01')
bt = Backtest(data, DualMomentumStrategy, cash=10000)
bt.run()

Start                                                   2024-01-02 00:00:00
End                                                     2025-05-02 00:00:00
Duration                                                  486 days 00:00:00
Exposure Time [%]                                                 69.850746
Equity Final [$]                                                9096.036009
Equity Peak [$]                                                11450.573417
Return [%]                                                         -9.03964
Buy & Hold Return [%]                                              8.250208
Return (Ann.) [%]                                                  -9.66095
Volatility (Ann.) [%]                                             19.243615
Sharpe Ratio                                                      -0.502034
Sortino Ratio                                                     -0.626171
Calmar Ratio                                                      -0.357904
Max. Drawdow

In [6]:
bt.plot(plot_allocation=True)

GridPlot(id='p2659', ...)